In [4]:
# TODO - naming, state, key...

class StateException(Exception):
    pass

class State:
    def __init__(self, state_store, key, state, *args, **kwargs):
        self.key = key
        self.state = state
        self.state_store = state_store
        self.state_store.update(key, state)
    
    def __repr__(self):
        return f"<{self.__class__.__name__} {self.state}>"

class ContextManagerState(State):
    """
    :param cm_raises_exception: None if use as a context manager is allowed.
    :param cm_raises_exception: (exception, args, kwargs) 
    exception to raise on use as context manager.
    """
    def __init__(self, state_store, key, state, 
                 cm_raises_exception=None, 
                 *args, **kwargs):
        self.cm_raises_exception = cm_raises_exception
        self.entered = False

        if cm_raises_exception:
            self.restore_state = None
        else:
            # restore state for when used as context manager.
            self.restore_state = dict(state_store.states[key])
        
        State.__init__(self, state_store, key, state, **kwargs)
    
    def __enter__(self):
        """
        Context manager.
        
        Throw exception if self.cm_raises_exception is set.
        
        Set state back to restore state, then push new color onto stack.
        """
        if self.cm_raises_exception:
            ex = self.cm_raises_exception[0]
            
            if len(self.cm_raises_exception) == 1:
                raise ex()
                
            args = self.cm_raises_exception[1]
            if len(self.cm_raises_exception) == 2:
                raise ex(*args)
                
            kwargs = self.cm_raises_exception[2]
            raise ex(*args, **kwargs)
            
        # Avoid expensive list insert, by resetting state + pushing new state. 
        self.state_store.update(self.key, self.restore_state)
        self.state_store.push(self.key, self.state)
        
    def __exit__(self, *args):
        self.state_store.pop()
        self.state = self.restore_state


class ColorState(ContextManagerState):
    def __init__(self, state_store, key, state, mode, value, **kwargs):
        """
        :param state_store: Backend to store color data.
        :param mode: rgba, indexed
        :param value: channel data
        """
        ContextManagerState.__init__(self, state_store, key,
                                     {"mode": mode, "value": value},
                                     **kwargs)
        
class BlendModeState(ContextManagerState):
    def __init__(self, state_store, key, state, blendmode, **kwargs):
        """
        :param state_store: Backend to store color data.
        :param blendmode: rgba, indexed
        """
        ContextManagerState.__init__(self, state_store, key,
                                     {"blendmode": blendmode})


class PathState(State):
    def __init__(self, state_store, fill, stroke):
        State.__init__(self, state_store, "path",
                       {"fill": fill, "stroke": stroke,
                       "elements": []},
                       cm_raises_exception)
        
    @property
    def elements(self):
        return self.state["elements"]


class StateStore:
    def __init__(self, states):
        self.stack = []
        self.stack.append(states)
            
    def update(self, key, state):
        self.states[key].update(state)
        return len(self.stack) -1
    
    @property
    def states(self):
        return self.stack[-1]
    
    def pop(self):
        del self.stack[-1]

    def push(self, key, state):
        states = {**self.states, **{key: {**state}}}
        self.stack.append(states)
        return len(self.stack) -1
    
    def __repr__(self):
        return f"<StateStack {self.stack}>"
        

class Context:
    def __init__(self):
        default_state = dict(
            fill={"mode": "rgba", "value": (1., 1., 1., 1.)},
            stroke={"mode": "rgba", "value": (0., 0., 0., 1.)},
        )
        self.state_store = StateStore(default_state)

    def fill(self, *rgba):
        if not rgba:
            return ColorState(self.state_store,
                              "fill",
                              self.state_store.states["fill"],
                              **self.state_store.states["fill"],
                             #cm_raises_exception=(ValueError, ["Not allowed here."])
                             )
        color = ColorState(self.state_store, "fill",
                           self.state_store.states["fill"],
                           "rgba", rgba,
                          #cm_raises_exception=(StateException, ["Cannot use a Context Manager without first specifying a color."])
                          )
        return color
    
    def __repr__(self):
        return f"<Context {self.state_store}>"
    
ctx = Context()
#print(ctx.state_store.stack[-1])
#color = ctx.fill(1., 0., 1., 1.)
print(ctx.state_store.states)
print()
ctx.fill()
#print(ctx)
with ctx.fill(1.0, 0, 0, 1.0):
    print("Inside CM")
    print(ctx.state_store.states)
    
print()
print(ctx.state_store.states)

{'fill': {'mode': 'rgba', 'value': (1.0, 1.0, 1.0, 1.0)}, 'stroke': {'mode': 'rgba', 'value': (0.0, 0.0, 0.0, 1.0)}}

Inside CM
{'fill': {'mode': 'rgba', 'value': (1.0, 0, 0, 1.0)}, 'stroke': {'mode': 'rgba', 'value': (0.0, 0.0, 0.0, 1.0)}}

{'fill': {'mode': 'rgba', 'value': (1.0, 1.0, 1.0, 1.0)}, 'stroke': {'mode': 'rgba', 'value': (0.0, 0.0, 0.0, 1.0)}}


In [ ]:
# Need nested state

#fill=ColorState

state={"fill": {"mode": "rgba", "value", [1,.2,.1, 1]}}